# EJERCICIO

### *IMPORTO BIBLIOTECAS*

In [5]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Corrección aquí
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, SeparableConv2D, Flatten, Dense, Input, Dropout, BatchNormalization, Lambda, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.optimizers import Adam 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np


2025-04-15 07:39:59.732182: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 07:39:59.794179: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 07:40:00.077403: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 07:40:00.188373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744702800.414828    5868 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744702800.50

### *CARGO LOS DATOS*

In [8]:
# Generador de imágenes con separación de validación
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% de las imágenes de entrenamiento se usarán para validación
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Cargar imágenes de entrenamiento (80% de train/)

    directory="/workspaces/233-PILAR-/data/train",
    target_size=(200train_generator = train_datagen.flow_from_directory(, 200),
    batch_size=32,
    class_mode='binary',
    subset="training"  # Indicar que son imágenes de entrenamiento
)

# Cargar imágenes de validación (20% de train/)
validation_generator = train_datagen.flow_from_directory(
    directory="/workspaces/TensorFlow-Pet-Classifier/data/train",
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary',
    subset="validation"  # Indicar que son imágenes de validación
)


IndentationError: unexpected indent (1697047779.py, line 12)

In [ ]:
# Definir el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPool2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Clasificación binaria (perro o gato)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


NameError: name 'Sequential' is not defined

In [ ]:
# Callbacks para mejorar el entrenamiento
checkpoint = ModelCheckpoint("/workspaces/TensorFlow-Pet-Classifier/models/best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

In [ ]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint, early_stop, reduce_lr]
)


In [ ]:

# Extraer métricas del entrenamiento
history_dict = history.history
epochs_range = range(len(history_dict['loss']))

# Crear la figura
plt.figure(figsize=(12, 5))

# Gráfica de la pérdida (Loss)
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history_dict['loss'], label='Pérdida en entrenamiento')
plt.plot(epochs_range, history_dict['val_loss'], label='Pérdida en validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Evolución de la Pérdida')

# Gráfica de la precisión (Accuracy)
plt.subplot(1, 2, 2)
plt.plot(epochs_range, history_dict['accuracy'], label='Precisión en entrenamiento')
plt.plot(epochs_range, history_dict['val_accuracy'], label='Precisión en validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.title('Evolución de la Precisión')

# Mostrar la figura
plt.tight_layout()
plt.show()


In [ ]:
# Función para predecir y mostrar múltiples imágenes
def predict_and_plot_grid(model, img_paths, threshold=0.5):
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))  # 2 filas, 5 columnas
    
    for ax, img_path in zip(axes.flatten(), img_paths):
        img = image.load_img(img_path, target_size=(200, 200))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

        prediction = model.predict(img_array)
        pred_label = "Perro " if prediction[0][0] > threshold else "Gato "
        prob = float(prediction[0][0]) if pred_label == "Perro " else 1 - float(prediction[0][0])

        img_bgr = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        ax.imshow(img_rgb)
        ax.set_title(f"{pred_label}\n({prob*100:.2f}%)")
        ax.axis("off")

    plt.tight_layout()
    plt.show()

# Evaluar 10 imágenes aleatorias de prueba
test_directory = "/workspaces/TensorFlow-Pet-Classifier/data/test"
test_images = [os.path.join(test_directory, fname) for fname in os.listdir(test_directory) if fname.endswith('.jpg') or fname.endswith('.png')]

# Seleccionar 10 imágenes aleatorias
random_images = np.random.choice(test_images, 10, replace=False)
predict_and_plot_grid(model, random_images)
